# Creating STAC Catalogs for Contextual Data

In [1]:
from pathlib import Path
from datetime import datetime

# pystac
from pystac import Catalog, Item, Asset, CatalogType

# dask_geopandas, dask-expr, pyogrio
import dask_geopandas as dg

import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import MultiPoint

## Create a STAC Catalog from scratch

### Create a new, empty catalog

In [2]:
catalog = Catalog(id="contextual", description="Contextual dataset")
path_root = Path("./data/catalog/")
catalog.normalize_and_save(
    root_href=path_root.absolute().resolve().as_posix(),
    catalog_type=CatalogType.SELF_CONTAINED,
)

### Make a STAC catalog for BAG data

In [3]:
path_BAG = Path('./data/bag.parquet')

In [4]:
data_BAG = dg.read_parquet(path_BAG)
data_BAG

,rdf_seealso,identificatie,bouwjaar,status,gebruiksdoel,oppervlakte_min,oppervlakte_max,aantal_verblijfsobjecten,geometry
npartitions=50,,,,,,,,,
,string,string,int64,string,string,float64,float64,int64,geometry
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [5]:
def get_metadata_bag(ddf):

    metadata = dict()
    
    # Metadata can be retrieved from the delayed object
    metadata["columns"] = ddf.columns.tolist()

    # Metadata can be retrieved from the first partition
    gs_part0 = ddf.get_partition(0).compute()
    metadata["epsg_code"] = gs_part0["geometry"].crs.to_epsg()
    
    # Metadata needs to be retrieved from all partitions
    # For now iterate over all partitions, improvement from dask-geopandas WIP 
    arr_bbox = np.empty((0, 4))
    list_geom_types = []
    for i in range(ddf.npartitions):
        gs = ddf.get_partition(i)["geometry"].compute()
        arr_bbox = np.vstack((arr_bbox, gs.total_bounds))
        list_geom_types = list_geom_types + gs.geom_type.unique().tolist()
    
    # Get the bounding box
    bbox = [
        arr_bbox[:, 0].min(),
        arr_bbox[:, 1].min(),
        arr_bbox[:, 2].max(),
        arr_bbox[:, 3].max(),
    ]

    # Get unique geometry types
    geom_types = list(set(list_geom_types))

    metadata["bbox"] = bbox
    metadata["geometry_types"] = geom_types

    return metadata

In [6]:
metadata = get_metadata_bag(data_BAG)
metadata

{'columns': ['rdf_seealso',
  'identificatie',
  'bouwjaar',
  'status',
  'gebruiksdoel',
  'oppervlakte_min',
  'oppervlakte_max',
  'aantal_verblijfsobjecten',
  'geometry'],
 'epsg_code': 28992,
 'bbox': [13603.33, 306900.396, 277924.306, 617112.488],
 'geometry_types': ['Polygon']}

In [7]:
# Create a new Item
item = Item(
    id="bag",
    geometry=None,
    bbox=metadata["bbox"],  # Bounding box of the dataset
    datetime=datetime.strptime(
        "2024-03-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ"
    ),  # datatime of the dataset, from https://service.pdok.nl/lv/bag/atom/bag.xml
    properties={
        "columns": metadata["columns"],
        "geometry_types": metadata["geometry_types"],
        "reference_system": metadata["epsg_code"],
    },
)

# Add the Datacube Extension to the Item
item.stac_extensions.append("datacube")

# Add path to the BAG dataset as an asset
asset = Asset(href=path_BAG.absolute().resolve().as_posix())
item.add_asset(key="data", asset=asset)

item

<Item id=bag>

In [8]:
# Update the catalog with the new item
catalog = Catalog.from_file(path_root / 'catalog.json')
catalog.add_item(item)

<Link rel=item target=<Item id=bag>>

In [9]:
catalog.normalize_and_save(
    root_href=path_root.absolute().resolve().as_posix(),
    catalog_type=CatalogType.SELF_CONTAINED,
)

### Make a STAC catalog for KNMI data

In [13]:
path_knmi = Path('./data/knmi.zarr')
path_knmi.exists()

True

In [14]:
ds_knmi = xr.open_zarr(path_knmi)
ds_knmi

<xarray.Dataset>
Dimensions:  (station: 63, time: 965)
Coordinates:
  * station  (station) int64 201 203 204 205 206 207 ... 356 370 375 377 380 391
  * time     (time) datetime64[ns] 2008-01-01 2008-01-01 ... 2008-02-13
Data variables: (12/32)
    DD       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    DR       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FF       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FH       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    FX       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    HH       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    ...       ...
    T10      (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    TD       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    U        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    VV       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    WW       (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>
    Y        (station, time) float64 dask.array<chunksize=(10, 965), meta=np.ndarray>

In [15]:
stations_geom = MultiPoint(
    [(x, y) for x, y in zip(ds_knmi["POS_X"], ds_knmi["POS_Y"])]
).__geo_interface__ # Convert to GeoJSON dict
stations_geom

{'type': 'MultiPoint',
 'coordinates': ((-4.5, 707.1),
  (15.7, 487.6),
  (37.6, 588.3),
  (55.0, 825.1),
  (65.1, 682.0),
  (126.7, 625.3),
  (191.8, 611.7),
  (95.9, 497.6),
  (-5.8, 651.3),
  (71.8, 548.5),
  (197.9, 672.5),
  (89.9, 461.6),
  (98.4, 497.4),
  (110.2, 556.5),
  (114.2, 548.9),
  (110.6, 763.5),
  (114.2, 481.1),
  (123.8, 583.7),
  (140.5, 516.0),
  (127.3, 517.2),
  (152.2, 600.4),
  (10.3, 583.3),
  (101.7, 502.1),
  (155.9, 517.7),
  (140.8, 456.6),
  (140.8, 456.6),
  (154.7, 545.4),
  (164.0, 496.5),
  (179.3, 581.8),
  (188.8, 523.8),
  (188.2, 451.8),
  (209.0, 603.0),
  (214.2, 494.3),
  (235.0, 529.6),
  (235.1, 571.3),
  (242.0, 453.9),
  (221.9, 621.2),
  (272.7, 580.0),
  (257.6, 477.1),
  (15.2, 378.6),
  (30.4, 385.0),
  (33.1, 421.2),
  (6.1, 392.6),
  (58.4, 384.9),
  (37.8, 408.7),
  (48.4, 360.5),
  (36.9, 438.8),
  (9.9, 447.5),
  (51.8, 394.8),
  (59.3, 401.5),
  (68.1, 445.4),
  (72.0, 388.3),
  (82.3, 384.7),
  (81.0, 434.2),
  (90.4, 441.7),
 

In [16]:
bbox = [
    ds_knmi["POS_X"].values.min(),
    ds_knmi["POS_Y"].values.min(),
    ds_knmi["POS_X"].values.max(),
    ds_knmi["POS_Y"].values.max(),
]
bbox

[-5.8, 323.9, 272.7, 825.1]

In [17]:
attrs_dict = {var_name: ds_knmi[var_name].attrs for var_name in ds_knmi.data_vars}
attrs_dict

{'DD': {'description': 'Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil 990=veranderlijk. Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;'},
 'DR': {'description': 'Duur van de neerslag (in 0.1 uur) per uurvak;'},
 'FF': {'description': 'Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur;'},
 'FH': {'description': 'Uurgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken;'},
 'FX': {'description': 'Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak;'},
 'HH': {'description': 'tijd (HH=uur, UT.12 UT=13 MET, 14 MEZT. Uurvak 05 loopt van 04.00 UT tot 5.00 UT;'},
 'HOOGTE': {},
 'IX': {'description': 'Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen, 2

In [18]:
# Create a new Item
item = Item(id='knmi',
            geometry=stations_geom,
            bbox=bbox,
            datetime=None,
            start_datetime=pd.Timestamp(ds_knmi['time'].min().values).to_pydatetime(),
            end_datetime=pd.Timestamp(ds_knmi['time'].min().values).to_pydatetime(),
            properties={"data_vars": list(ds_knmi.data_vars), 'geometry_types': 'Point'})

# Add the Datacube Extension to the Item
item.stac_extensions.append('datacube')

# Add the Datacube Extension properties to the Item
item.properties.update({'cube:variables': attrs_dict})

# Add path to the BAG dataset as an asset
asset = Asset(href=path_knmi.absolute().resolve().as_posix())
item.add_asset(key='data', asset=asset)

item

<Item id=knmi>

In [20]:
path_root = Path("./data/catalog/")
catalog = Catalog.from_file(path_root / 'catalog.json')
catalog

<Catalog id=contextual>

In [21]:
catalog.add_item(item)
catalog

<Catalog id=contextual>

In [22]:
catalog.normalize_and_save(path_root.absolute().resolve().as_posix(), catalog_type=CatalogType.SELF_CONTAINED)